<a href="https://colab.research.google.com/github/bebyakinb/test_assignments_Analyst/blob/master/habr/SQL_Hardest_Interview_Tasks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Подготовка среды

In [1]:
!wget https://www.sqlite.org/src/tarball/sqlite.tar.gz?r=release -O sqlite.tar.gz &> /dev/null
!tar xzf sqlite.tar.gz &> /dev/null
%cd sqlite/
!./configure &> /dev/null
!make sqlite3.c &> /dev/null
%cd /content
!npx degit coleifer/pysqlite3 -f &> /dev/null
!cp sqlite/sqlite3.[ch] .
!python setup.py build_static build &> /dev/null
!cp build/lib.linux-x86_64-3.8/pysqlite3/_sqlite3.cpython-38-x86_64-linux-gnu.so \
    /usr/lib/python3.8/lib-dynload/_sqlite3.cpython-38-x86_64-linux-gnu.so

/content/sqlite
/content


In [2]:
import sqlite3
sqlite3.sqlite_version

'3.40.0'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. Задача №1. Процентное изменение месяц к месяцу


## 1.0. Постановка задачи
**Контекст:** часто полезно знать, как изменяется ключевая метрика, например, месячная аудитория активных пользователей, от месяца к месяцу. Допустим у нас есть таблица logins в таком виде:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```
**Задача:** Найти ежемесячное процентное изменение месячной аудитории активных пользователей (MAU).

## 1.1. Подготовка данных

In [4]:
def random_dates(start, end, n=10):
    '''
    Dividing the unix time values(start and end) by 24*60*60*10**9
    to make it days and return n randomly chosen days back.
    '''
    start_u = start.value//(24*60*60*10**9)
    end_u = end.value//(24*60*60*10**9)

    return pd.to_datetime(np.random.randint(start_u, end_u, n), unit='D')


In [5]:
start = pd.to_datetime('2021-01-01')
end = pd.to_datetime('2022-12-01')
login_lines = 100000
user_ids = 30000

logins = pd.DataFrame()
logins['user_id'] = np.random.randint(1, user_ids, login_lines)
logins['date'] = pd.Series(random_dates(start, end, login_lines))
logins

,user_id,date
0,3071,2021-08-18
1,3608,2022-10-08
2,16702,2022-02-06
3,21967,2022-03-17
4,428,2021-01-26
...,...,...
99995,1480,2021-11-05
99996,12239,2022-03-21
99997,14625,2022-07-22
99998,20709,2022-08-29


In [6]:
con = sqlite3.connect('db')
cur = con.cursor()

In [7]:
logins.to_sql('logins', con, if_exists='replace',index=False)

In [8]:
def select(sql):
  return pd.read_sql(sql,con)

In [9]:
sql = '''select * from logins'''
select(sql)

,user_id,date
0,3071,2021-08-18 00:00:00
1,3608,2022-10-08 00:00:00
2,16702,2022-02-06 00:00:00
3,21967,2022-03-17 00:00:00
4,428,2021-01-26 00:00:00
...,...,...
99995,1480,2021-11-05 00:00:00
99996,12239,2022-03-21 00:00:00
99997,14625,2022-07-22 00:00:00
99998,20709,2022-08-29 00:00:00


## 1.2. Решение SQL

In [10]:
sql ='''
with months_active as (
  select strftime('%Y %m', date) as year_month,
         cast(count(user_id) as real) as users from logins
  group by year_month)

select 
year_month, users, (users - LAG(users) over(order by year_month))*100/LAG(users) over(order by year_month) MAU_percent
from months_active
'''
select(sql)

,year_month,users,MAU_percent
0,2021 01,4446.0,NaN
1,2021 02,3924.0,-11.740891
2,2021 03,4463.0,13.735984
3,2021 04,4335.0,-2.868026
4,2021 05,4373.0,0.876586
5,2021 06,4306.0,-1.532129
6,2021 07,4430.0,2.879703
7,2021 08,4405.0,-0.564334
8,2021 09,4359.0,-1.044268
9,2021 10,4396.0,0.848819


## 1.3 Решение Pandas

In [11]:
pt = logins.pivot_table(index=[(logins['date'].dt.to_period('M'))], values=['user_id'], aggfunc="count")
pt['MAU,%mm'] = (pt['user_id'] - pt['user_id'].shift(1))/pt['user_id'].shift(1)*100
pt

,user_id,"MAU,%mm"
date,,
2021-01,4446,NaN
2021-02,3924,-11.740891
2021-03,4463,13.735984
2021-04,4335,-2.868026
2021-05,4373,0.876586
2021-06,4306,-1.532129
2021-07,4430,2.879703
2021-08,4405,-0.564334
2021-09,4359,-1.044268


# 2.Задача №2. Маркировка древовидной структуры

## 2.0 Постановка задачи
Контекст: предположим, у вас есть таблица tree с двумя столбцами: в первом указаны узлы, а во втором — родительские узлы.
```
node   parent
1       2
2       5
3       5
4       3
5       NULL 
```
Задача: написать SQL таким образом, чтобы мы обозначили каждый узел как внутренний (inner), корневой (root) или конечный узел/лист (leaf), так что для вышеперечисленных значений получится следующее:
```
node    label  
1       Leaf
2       Inner
3       Inner
4       Leaf
5       Root
```

## 2.1 Подготовка данных

In [12]:
nodes = pd.DataFrame({'node': [1,2,3,4,5],
                      'parent':[2,5,5,3,np.nan]})
nodes

,node,parent
0,1,2.0
1,2,5.0
2,3,5.0
3,4,3.0
4,5,NaN


In [13]:
nodes.to_sql('nodes', con, if_exists='replace',index=False)

### 2.2 Решение SQL

In [14]:
sql = '''
select node,
       case
            when parent is null then 'Root'
            when node in (select parent from nodes) then 'Inner'
            else 'Leaf'
       end as label
from nodes
'''
select(sql)

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


## 2.3 Решение Pandas

In [15]:
nodes['label'] =  np.where(nodes['parent'].isna(), 'Root',
                  np.where(nodes['node'].isin(nodes['parent']), 'Inner', 'Leaf'))
nodes[['node','label']]

,node,label
0,1,Leaf
1,2,Inner
2,3,Inner
3,4,Leaf
4,5,Root


# 3. Задача № 3. Удержание пользователей в месяц.

## 3.0. Постановка задачи

**Контекст:** допустим, у нас есть статистика по авторизации пользователей на сайте в таблице logins:
```
| user_id | date       |
|---------|------------|
| 1       | 2018-07-01 |
| 234     | 2018-07-02 |
| 3       | 2018-07-02 |
| 1       | 2018-07-02 |
| ...     | ...        |
| 234     | 2018-10-04 |
```

**Задача:** написать запрос, который получает количество удержанных пользователей в месяц. В нашем случае данный параметр определяется как количество пользователей, которые авторизовались в системе и в этом, и в предыдущем месяце.


## 3.1. Подготовка данных

In [16]:
logins.to_sql('logins', con, if_exists='replace',index=False)

## 3.2. Решение SQL

In [17]:
sql = '''
with months_logins as (
  select distinct user_id, 
         strftime('%Y %m', date) as cur_month,
         strftime('%Y %m', date(date, 'start of month','-1 month')) as prev_month
  from logins
  )

select m1.cur_month, count(m2.user_id)
from months_logins as m1
left join months_logins as m2 ON m1.user_id = m2.user_id AND m1.prev_month = m2.cur_month
group by m1.cur_month
'''
select(sql)

,cur_month,count(m2.user_id)
0,2021 01,0
1,2021 02,492
2,2021 03,481
3,2021 04,560
4,2021 05,542
5,2021 06,550
6,2021 07,545
7,2021 08,551
8,2021 09,548
9,2021 10,544


## 3.3. Решение Pandas

In [18]:
logins_extended = logins.copy()
logins_extended['id_and_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + logins['date'].dt.to_period('M').dt.to_timestamp().astype(str))
logins_extended['id_and_prev_month'] = (
    logins['user_id'].astype(str) 
    + '_' 
    + (logins['date'].dt.to_period('M').dt.to_timestamp() 
       - pd.DateOffset(months=1)).astype(str))
logins_extended = logins_extended.drop_duplicates(['user_id','id_and_month'])
logins_extended

,user_id,date,id_and_month,id_and_prev_month
0,3071,2021-08-18,3071_2021-08-01,3071_2021-07-01
1,3608,2022-10-08,3608_2022-10-01,3608_2022-09-01
2,16702,2022-02-06,16702_2022-02-01,16702_2022-01-01
3,21967,2022-03-17,21967_2022-03-01,21967_2022-02-01
4,428,2021-01-26,428_2021-01-01,428_2020-12-01
...,...,...,...,...
99994,18955,2022-11-27,18955_2022-11-01,18955_2022-10-01
99995,1480,2021-11-05,1480_2021-11-01,1480_2021-10-01
99997,14625,2022-07-22,14625_2022-07-01,14625_2022-06-01
99998,20709,2022-08-29,20709_2022-08-01,20709_2022-07-01


In [19]:
logins_extended['prev_month_login'] = (
    np.where(logins_extended['id_and_prev_month'].isin(logins_extended['id_and_month'])
             , 1 , 0))
logins_extended.groupby([logins_extended['date'].dt.to_period('M')])['prev_month_login'].sum()

<ipython-input-19-51ef2d549242>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  logins_extended['prev_month_login'] = (


date
2021-01      0
2021-02    492
2021-03    481
2021-04    560
2021-05    542
2021-06    550
2021-07    545
2021-08    551
2021-09    548
2021-10    544
2021-11    550
2021-12    575
2022-01    601
2022-02    538
2022-03    509
2022-04    539
2022-05    512
2022-06    555
2022-07    562
2022-08    568
2022-09    599
2022-10    558
2022-11    576
Freq: M, Name: prev_month_login, dtype: int64

# 4. Задача № 4. Нарастающий итог

## 4.0. Поставнока задачи

**Контекст:** допустим, у нас есть таблица transactions в таком виде:

| date       | cash_flow |
|------------|-----------|
| 2018-01-01 | -1000     |
| 2018-01-02 | -100      |
| 2018-01-03 | 50        |
| ...        | ...       |


Где cash_flow — это выручка минус затраты за каждый день.

**Задача:** написать запрос, чтобы получить нарастающий итог для денежного потока каждый день таким образом, чтобы в конечном итоге получилась таблица в такой форме:

| date       | cumulative_cf |
|------------|---------------|
| 2018-01-01 | -1000         |
| 2018-01-02 | -1100         |
| 2018-01-03 | -1050         |
| ...        | ...           |

## 4.1. Подготовка данных

In [99]:
sample_size = 10
start = pd.to_datetime('2021-01-01')

transactions = pd.DataFrame()
transactions['date'] = [start + pd.DateOffset(days=i) for i in range(sample_size)]
transactions['cash_flow'] = np.random.randint(-2000, 2000, sample_size)
transactions

,date,cash_flow
0,2021-01-01,-104
1,2021-01-02,1540
2,2021-01-03,1268
3,2021-01-04,-859
4,2021-01-05,1201
5,2021-01-06,392
6,2021-01-07,-1554
7,2021-01-08,-3
8,2021-01-09,1117
9,2021-01-10,-334


In [100]:
transactions.to_sql('transactions', con, if_exists='replace',index=False)

## 4.2. Решение SQL

In [101]:
sql = '''
select
  date,
  sum(cash_flow) over (order by date) AS cumulative_cf
from transactions
group by date
order by date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,-104
1,2021-01-02 00:00:00,1436
2,2021-01-03 00:00:00,2704
3,2021-01-04 00:00:00,1845
4,2021-01-05 00:00:00,3046
5,2021-01-06 00:00:00,3438
6,2021-01-07 00:00:00,1884
7,2021-01-08 00:00:00,1881
8,2021-01-09 00:00:00,2998
9,2021-01-10 00:00:00,2664


## 4.3. Решение SQL без оконной функции

In [102]:
sql = '''
select
  t1.date,
  sum(t2.cash_flow) as cumulative_cf
from transactions as t1
join transactions as t2 on t1.date >= t2.date
group by t1.date
order by t1.date asc
'''
select(sql)

,date,cumulative_cf
0,2021-01-01 00:00:00,-104
1,2021-01-02 00:00:00,1436
2,2021-01-03 00:00:00,2704
3,2021-01-04 00:00:00,1845
4,2021-01-05 00:00:00,3046
5,2021-01-06 00:00:00,3438
6,2021-01-07 00:00:00,1884
7,2021-01-08 00:00:00,1881
8,2021-01-09 00:00:00,2998
9,2021-01-10 00:00:00,2664


## 4.4. Решение Pandas

In [103]:
transactions_extended = transactions.copy().sort_values('date')
transactions_extended['cumulative_cf'] = transactions_extended['cash_flow'].cumsum()
transactions_extended[['date','cumulative_cf']]

,date,cumulative_cf
0,2021-01-01,-104
1,2021-01-02,1436
2,2021-01-03,2704
3,2021-01-04,1845
4,2021-01-05,3046
5,2021-01-06,3438
6,2021-01-07,1884
7,2021-01-08,1881
8,2021-01-09,2998
9,2021-01-10,2664


**Контекст:** допустим, у нас есть таблица signups в таком виде:

| date       | sign_ups |
|------------|----------|
| 2018-01-01 | 10       |
| 2018-01-02 | 20       |
| 2018-01-03 | 50       |
| ...        | ...      |
| 2018-10-01 | 35       |


**Задача:** написать запрос, чтобы получить 7-дневное скользящее среднее ежедневных регистраций